In [56]:
import pandas as pd

In [57]:
df = pd.read_csv("../data/parsed_logs_fi.csv")

In [58]:
df["config"].unique()

array(['c10_res44_test_01_bn-relu_base', 'c10_res44_test_02_bn-relu6',
       'c10_res44_test_02_bn-relu6_base', 'c10_res44_test_02_relu6-bn',
       'c10_res44_test_02_relu6-bn_nanfilter',
       'c10_res44_test_02_gelu6_nans', 'c100_res44_test_01_bn-relu_base',
       'c100_res44_test_02_bn-relu6', 'c100_res44_test_02_bn-relu6_base',
       'c100_res44_test_02_relu6-bn',
       'c100_res44_test_02_relu6-bn_nanfilter',
       'c100_res44_test_02_gelu6_nans'], dtype=object)

In [59]:
configs = dict(
    # Baseline C100
    BaseC100="c100_res44_test_01_bn-relu_base.yaml",
    # "c100_res44_test_02_bn-relu6.yaml",
    # Order inversion with relu6 C100
    OrderInvC100="c100_res44_test_02_relu6-bn.yaml",
    # "c100_res44_test_02_bn-relu6_base.yaml",
    BaseBnRelu6C100="c100_res44_test_02_bn-relu6_base.yaml",
    BnRelu6C100="c100_res44_test_02_bn-relu6.yaml",
    # Order inversion + nan filter + Relu6
    OrderINanRelu6C100="c100_res44_test_02_relu6-bn_nanfilter.yaml",
    # Gelu and nan C100
    GeluNanC100="c100_res44_test_02_gelu6_nans.yaml",

    # Baseline C10
    BaseC10="c10_res44_test_01_bn-relu_base.yaml",
    # "c10_res44_test_02_bn-relu6.yaml",
    BnRelu6C10="c10_res44_test_02_bn-relu6.yaml",
    # Order inversion with relu6 C10
    OrderInvC10="c10_res44_test_02_relu6-bn.yaml",
    # "c10_res44_test_02_bn-relu6_base.yaml",
    BaseBnRelu6C10="c10_res44_test_02_bn-relu6_base.yaml",
    OrderINanRelu6C10="c10_res44_test_02_relu6-bn_nanfilter.yaml",
    # Gelu and nan C10
    GeluNanC10="c10_res44_test_02_gelu6_nans.yaml",

    MobileNetC100="cifar100_mobilenetv2_x1_4.yaml",
    MobileNetC10="cifar10_mobilenetv2_x1_4.yaml",

    Resnet50ImagenetBase="imagenet1k_v2_base"
)
configs_inv = {v.replace(".yaml", ""): k for k, v in configs.items()}

In [60]:
configs_df = pd.DataFrame([configs]).T.reset_index().rename(columns={0: "YAML", "index": "config"})

In [61]:
configs_df

,config,YAML
0,BaseC100,c100_res44_test_01_bn-relu_base.yaml
1,OrderInvC100,c100_res44_test_02_relu6-bn.yaml
2,BaseBnRelu6C100,c100_res44_test_02_bn-relu6_base.yaml
3,BnRelu6C100,c100_res44_test_02_bn-relu6.yaml
4,OrderINanRelu6C100,c100_res44_test_02_relu6-bn_nanfilter.yaml
5,GeluNanC100,c100_res44_test_02_gelu6_nans.yaml
6,BaseC10,c10_res44_test_01_bn-relu_base.yaml
7,BnRelu6C10,c10_res44_test_02_bn-relu6.yaml
8,OrderInvC10,c10_res44_test_02_relu6-bn.yaml
9,BaseBnRelu6C10,c10_res44_test_02_bn-relu6_base.yaml


In [62]:
df["config"] = df["config"].apply(lambda x: configs_inv[x])

In [63]:
df["count"] = 1

In [64]:
df[df["sdc"] == 0]

,start_dt,config,ecc,hostname,logfile,batch_size,it,ker_time,acc_time,ker_err,acc_err,sdc,critical_sdc,fi_model,group,count


In [65]:
to_save = df.groupby(["hostname", "config", "fi_model"]).sum()[["sdc", "critical_sdc", "count"]].reset_index()

In [66]:
NEW_NAMES = dict(
    # Baseline C100
    BaseC100="Base",
    # "c100_res44_test_02_bn-relu6.yaml",
    # Order inversion with relu6 C100
    OrderInvC100="OrdInv",
    # "c100_res44_test_02_bn-relu6_base.yaml",
    BaseBnRelu6C100="BaseR6",
    BnRelu6C100="BnR6",
    # Order inversion + nan filter + Relu6
    OrderINanRelu6C100="OrdInvNanR6",
    # Gelu and nan C100
    GeluNanC100="Gelu6Nan",
)
NEW_NAMES.update({k.replace("C100", "C10"): v for k, v in NEW_NAMES.items()})
NEW_NAMES["Resnet50ImagenetBase"] = "Resnet50ImagenetBase"

In [67]:
to_save

,hostname,config,fi_model,sdc,critical_sdc,count
0,caroltitaninria,BaseBnRelu6C10,0,124,1,124
1,caroltitaninria,BaseBnRelu6C10,2,179,2,179
2,caroltitaninria,BaseBnRelu6C10,3,158,0,158
3,caroltitaninria,BaseBnRelu6C10,4,246,12,246
4,caroltitaninria,BaseBnRelu6C10,5,251,0,251
...,...,...,...,...,...,...
85,caroltitantn,OrderInvC100,0,152,0,152
86,caroltitantn,OrderInvC100,2,224,8,224
87,caroltitantn,OrderInvC100,3,193,0,193
88,caroltitantn,OrderInvC100,4,308,37,308


In [68]:
FM_NAMES = dict(FLIP_SINGLE_BIT="Single Bit", ZERO_VALUE="Zero Val", WARP_ZERO_VALUE="Warp Z.V.",
                RANDOM_VALUE="Rand Val", WARP_RANDOM_VALUE="Warp R. V.")
FM_IDS = dict(FLIP_SINGLE_BIT=0, RANDOM_VALUE=2, ZERO_VALUE=3, WARP_RANDOM_VALUE=4, WARP_ZERO_VALUE=5)
FM_ID_TO_NAMES = {fmv: FM_NAMES[fmk] for fmk, fmv in FM_IDS.items()}

In [69]:
def set_dataset(x):
    if "C100" in x:
        return "Cifar 100"
    elif "Imagenet" in x:
        return "Imagenet"
    else:
        return "Cifar 10"

In [70]:
to_save["dataset"] = to_save["config"].apply(set_dataset)
to_save["config"] = to_save["config"].apply(lambda x: NEW_NAMES[x])
to_save["fi_model"] = to_save["fi_model"].apply(lambda x: FM_ID_TO_NAMES[x])

In [71]:
to_save

,hostname,config,fi_model,sdc,critical_sdc,count,dataset
0,caroltitaninria,BaseR6,Single Bit,124,1,124,Cifar 10
1,caroltitaninria,BaseR6,Rand Val,179,2,179,Cifar 10
2,caroltitaninria,BaseR6,Zero Val,158,0,158,Cifar 10
3,caroltitaninria,BaseR6,Warp R. V.,246,12,246,Cifar 10
4,caroltitaninria,BaseR6,Warp Z.V.,251,0,251,Cifar 10
...,...,...,...,...,...,...,...
85,caroltitantn,OrdInv,Single Bit,152,0,152,Cifar 100
86,caroltitantn,OrdInv,Rand Val,224,8,224,Cifar 100
87,caroltitantn,OrdInv,Zero Val,193,0,193,Cifar 100
88,caroltitantn,OrdInv,Warp R. V.,308,37,308,Cifar 100


In [72]:
to_save_tail = to_save.groupby(["hostname", "dataset", "config"]).sum().reset_index()

In [73]:
to_save_tail["fi_model"] = "Overall"

In [74]:
to_save_gp = pd.concat([to_save, to_save_tail])

In [75]:
to_save_gp = to_save_gp.groupby(["hostname", "dataset", "fi_model", "config"]).sum()

In [76]:
to_save_gp["avf sdc"] = (to_save_gp["sdc"] - to_save_gp["critical_sdc"]) / to_save_gp["count"]
to_save_gp["avf critical sdc"] = to_save_gp["critical_sdc"] / to_save_gp["count"]

In [77]:
to_save_gp

sdc  critical_sdc  count  \
hostname        dataset   fi_model config                                   
caroltitaninria Cifar 10  Overall  Base          927           213    927   
                                   BaseR6        958            15    958   
                                   BnR6          904            11    904   
                                   Gelu6Nan     1131            15   1131   
                                   OrdInv        855             7    855   
...                                              ...           ...    ...   
caroltitantn    Cifar 100 Zero Val BaseR6        205             0    205   
                                   BnR6          183             0    183   
                                   Gelu6Nan      290             0    290   
                                   OrdInv        193             0    193   
                                   OrdInvNanR6   200             0    200   

                                                 avf sdc  avf critical sdc  
hostname        dataset   fi_model config                                   
caroltitaninria Cifar 10  Overall  Base         0.770227          0.229773  
                                   BaseR6       0.984342          0.015658  
                                   BnR6         0.987832          0.012168  
                                   Gelu6Nan     0.986737          0.013263  
                                   OrdInv       0.991813          0.008187  
...                                                  ...               ...  
caroltitantn    Cifar 100 Zero Val BaseR6       1.000000          0.000000  
                                   BnR6         1.000000          0.000000  
                                   Gelu6Nan     1.000000          0.000000  
                                   OrdInv       1.000000          0.000000  
                                   OrdInvNanR6  1.000000          0.000000  

[108 rows x 5 columns]